In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [59]:
banks = ['JPM/jpmorgan-chase','GS/goldman-sachs','BAC/bank-of-america','C/citigroup','MS/morgan-stanley']
revenue_url = 'https://www.macrotrends.net/stocks/charts/{}/revenue'
roi_url = 'https://www.macrotrends.net/stocks/charts/{}/roi'

def get_url(url):
    urls = []
    for i in banks:
        urls.append(url.format(i))
        
    return urls

get_url(roi_url)

['https://www.macrotrends.net/stocks/charts/JPM/jpmorgan-chase/roi',
 'https://www.macrotrends.net/stocks/charts/GS/goldman-sachs/roi',
 'https://www.macrotrends.net/stocks/charts/BAC/bank-of-america/roi',
 'https://www.macrotrends.net/stocks/charts/C/citigroup/roi',
 'https://www.macrotrends.net/stocks/charts/MS/morgan-stanley/roi']

In [83]:
rev = []
for url, bank in zip(get_url(revenue_url), banks):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')[1]
    data = []
    table_rows = table.find_all('tr')
    for row in table_rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            data.append([cell.text.strip() for cell in cells])
    df = pd.DataFrame(data, columns=['Date', 'Revenue'])
    df['Bank'] = bank
    rev.append(df)

df_rev = pd.concat(rev, ignore_index=True)


In [87]:
df_rev

,Date,Revenue,Bank
0,2023-03-31,"$54,642",JPM/jpmorgan-chase
1,2022-12-31,"$47,409",JPM/jpmorgan-chase
2,2022-09-30,"$40,809",JPM/jpmorgan-chase
3,2022-06-30,"$34,233",JPM/jpmorgan-chase
4,2022-03-31,"$32,341",JPM/jpmorgan-chase
...,...,...,...
280,2010-03-31,"$9,072",MS/morgan-stanley
281,2009-12-31,"$6,718",MS/morgan-stanley
282,2009-09-30,"$8,468",MS/morgan-stanley
283,2009-06-30,"$5,197",MS/morgan-stanley


In [81]:
roi = []
for url, bank in zip(get_url(roi_url), banks):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    data = []
    table_rows = table.find_all('tr')
    for row in table_rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            data.append([cell.text.strip() for cell in cells])
    df_roi = pd.DataFrame(data, columns=['Date', 'TTM Net Income', 'LT Investments & Debt', 'ROI'])
    df_roi['Bank'] = bank
    roi.append(df_roi)

df_roi = pd.concat(roi, ignore_index=True)


In [86]:
df_roi

,Date,TTM Net Income,LT Investments & Debt,ROI,Bank
0,2023-03-31,$52.07B,$598.57B,8.91%,JPM/jpmorgan-chase
1,2022-12-31,$46.17B,$588.20B,7.97%,JPM/jpmorgan-chase
2,2022-09-30,$45.59B,$575.49B,7.85%,JPM/jpmorgan-chase
3,2022-06-30,$47.70B,$574.36B,8.16%,JPM/jpmorgan-chase
4,2022-03-31,$51.93B,$579.14B,8.84%,JPM/jpmorgan-chase
...,...,...,...,...,...
265,2010-12-31,$6.20B,$257.86B,2.46%,MS/morgan-stanley
266,2010-09-30,$5.87B,$262.07B,2.36%,MS/morgan-stanley
267,2010-06-30,$6.56B,$241.97B,2.68%,MS/morgan-stanley
268,2010-03-31,$4.28B,$244.38B,1.75%,MS/morgan-stanley


In [88]:
banking = pd.merge(df_rev, df_roi, how='inner', on=['Date','Bank'])

In [89]:
banking

,Date,Revenue,Bank,TTM Net Income,LT Investments & Debt,ROI
0,2023-03-31,"$54,642",JPM/jpmorgan-chase,$52.07B,$598.57B,8.91%
1,2022-12-31,"$47,409",JPM/jpmorgan-chase,$46.17B,$588.20B,7.97%
2,2022-09-30,"$40,809",JPM/jpmorgan-chase,$45.59B,$575.49B,7.85%
3,2022-06-30,"$34,233",JPM/jpmorgan-chase,$47.70B,$574.36B,8.16%
4,2022-03-31,"$32,341",JPM/jpmorgan-chase,$51.93B,$579.14B,8.84%
...,...,...,...,...,...,...
265,2010-12-31,"$7,415",MS/morgan-stanley,$6.20B,$257.86B,2.46%
266,2010-09-30,"$6,780",MS/morgan-stanley,$5.87B,$262.07B,2.36%
267,2010-06-30,"$7,963",MS/morgan-stanley,$6.56B,$241.97B,2.68%
268,2010-03-31,"$9,072",MS/morgan-stanley,$4.28B,$244.38B,1.75%
